In [1]:
# Import necessary packages
import pandas as pd

In [2]:
# Setup dataframes by importing and opening data sets
schools_df = pd.read_csv("Resources/schools_complete.csv")
students_df = pd.read_csv("Resources/students_complete.csv")
# Check snapshot of both dataframes to know how to merge
# students_df.head()

In [3]:
# Check snapshot of both dataframes to know how to merge
#schools_df

In [4]:
# Merge dataframes together
complete_df = pd.merge(schools_df, students_df, on=["school_name"])

# Add a sorted size column for ease later
size_bins = [0, 1500, 3500, 4976]
size_names = ["Small (< 1500)", "Medium (1500-3500)", "Large (> 3500)"]
complete_df["School Size"] = pd.cut(complete_df["size"], bins=size_bins, labels=size_names, include_lowest=True)

# Adding a per student budget column and sorting it for ease later
complete_df["Per Student Budget"] = complete_df["budget"]/complete_df["size"]
funding_bins = [0, 585, 630, 645, 680]
funding_names = ["<$585", "$585-630", "$630-645", "$645-680"]
complete_df["Spending Ranges (Per Student)"] = pd.cut(complete_df["Per Student Budget"], bins=funding_bins, labels=funding_names, include_lowest=True)

# complete_df.head()

In [5]:
# Calculating the percentages passing reading and math and overall to be used in creating summary dataframe
passing_math = complete_df.loc[complete_df["math_score"] >= 70,:]
passing_reading = complete_df.loc[complete_df["reading_score"] >= 70,:]
passing_overall = complete_df.loc[(complete_df["reading_score"] >= 70) & (complete_df["math_score"] >= 70),:]

# Creating a dataframe that is summary statistics for the district at large by first creating a dictionary and converting to df
district_summary = {}
district_summary["Total Schools"] = schools_df.iloc[-1, 0] + 1
district_summary["Total Students"] = schools_df["size"].sum()
district_summary["Total Budget"] = schools_df["budget"].sum()
district_summary["Average Math Score"] = round(students_df["math_score"].mean(),2)
district_summary["Average Reading Score"] = round(students_df["reading_score"].mean(),2)
district_summary["% Passing Math"] = round(((passing_math["Student ID"].count())/district_summary["Total Students"])*100, 2)
district_summary["% Passing Reading"] = round(((passing_reading["Student ID"].count())/district_summary["Total Students"])*100, 2)
district_summary["% Passing Overall"] = round(((passing_overall["Student ID"].count())/district_summary["Total Students"])*100, 2)
district_sum_df = pd.DataFrame(district_summary, index=[0])
district_sum_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,15,39170,24649428,78.99,81.88,74.98,85.81,65.17


In [6]:
# Create a dataframe for school summary with all rows and relevant columns
    # Create seperate series and new dataframe using grouping by school
grouped_school = complete_df.groupby(["school_name"])
avg_math_school = round(grouped_school["math_score"].mean(),2)
avg_reading_school = round(grouped_school["reading_score"].mean(),2)
passing_math_grp = passing_math.groupby(["school_name"]).count()
passing_reading_grp = passing_reading.groupby(["school_name"]).count()
passing_overall_grp = passing_overall.groupby(["school_name"]).count()
new_index = schools_df.set_index("school_name")
passing_math_per = round((passing_math_grp["Student ID"]/new_index["size"])*100,2)
passing_reading_per = round((passing_reading_grp["Student ID"]/new_index["size"])*100,2)
passing_overall_per = round((passing_overall_grp["Student ID"]/new_index["size"])*100,2)
school_prelim = pd.DataFrame({"Average Math Score": avg_math_school, \
                              "Average Reading Score": avg_reading_school, \
                              "% Passing Math": passing_math_per, \
                              "% Passing Reading": passing_reading_per, \
                              "% Passing Overall": passing_overall_per})
school_2 = pd.merge(school_prelim, schools_df, on=["school_name"])
school_2["Per Student Budget"] = school_2["budget"]/school_2["size"]
school_org = school_2[["school_name", "type", "size", "budget", "Per Student Budget", "Average Math Score", \
                       "Average Reading Score", "% Passing Math", "% Passing Reading", "% Passing Overall"]]
school_ren = school_org.rename(columns={"school_name": "School Name", "type": "School Type", "size": "Total Students",\
                                        "budget": "Total Budget"})
school_ren["Total Budget"] = school_ren["Total Budget"].map("${:,.0f}".format)
school_ren["Per Student Budget"] = school_ren["Per Student Budget"].map("${:,.0f}".format)
school_final = school_ren.set_index(["School Name"])
school_final

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928",$628,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Charter,1858,"$1,081,356",$582,83.06,83.98,94.13,97.04,91.33
Figueroa High School,District,2949,"$1,884,411",$639,76.71,81.16,65.99,80.74,53.20
Ford High School,District,2739,"$1,763,916",$644,77.10,80.75,68.31,79.30,54.29
Griffin High School,Charter,1468,"$917,500",$625,83.35,83.82,93.39,97.14,90.60
Hernandez High School,District,4635,"$3,022,020",$652,77.29,80.93,66.75,80.86,53.53
Holden High School,Charter,427,"$248,087",$581,83.80,83.81,92.51,96.25,89.23
Huang High School,District,2917,"$1,910,635",$655,76.63,81.18,65.68,81.32,53.51
Johnson High School,District,4761,"$3,094,650",$650,77.07,80.97,66.06,81.22,53.54


In [7]:
# Sort the school-sorted dataframe by overall passing percentage
school_sorted = school_final.sort_values(by=["% Passing Overall"], ascending=False)
high_perf = school_sorted.head(5)
high_perf

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356",$582,83.06,83.98,94.13,97.04,91.33
Thomas High School,Charter,1635,"$1,043,130",$638,83.42,83.85,93.27,97.31,90.95
Griffin High School,Charter,1468,"$917,500",$625,83.35,83.82,93.39,97.14,90.60
Wilson High School,Charter,2283,"$1,319,574",$578,83.27,83.99,93.87,96.54,90.58
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59,95.95,90.54


In [8]:
# Find the lowest performing schools using sorted dataframe
low = school_sorted.tail(5)
low_perf = low.sort_values(by=["% Passing Overall"],)
low_perf

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363",$637,76.84,80.74,66.37,80.22,52.99
Figueroa High School,District,2949,"$1,884,411",$639,76.71,81.16,65.99,80.74,53.20
Huang High School,District,2917,"$1,910,635",$655,76.63,81.18,65.68,81.32,53.51
Hernandez High School,District,4635,"$3,022,020",$652,77.29,80.93,66.75,80.86,53.53
Johnson High School,District,4761,"$3,094,650",$650,77.07,80.97,66.06,81.22,53.54


In [9]:
# Create Dataframes sorted for each grade with columns for reading and math scores and school name
grade_9_full = complete_df.loc[complete_df["grade"] == "9th",["math_score", "reading_score", "school_name"]]
grade_10_full = complete_df.loc[complete_df["grade"] == "10th",["math_score", "reading_score", "school_name"]]
grade_11_full = complete_df.loc[complete_df["grade"] == "11th",["math_score", "reading_score", "school_name"]]
grade_12_full = complete_df.loc[complete_df["grade"] == "12th",["math_score", "reading_score", "school_name"]]

# Create dataframes grouped by school name using the dataframes sorted by grade
grade_9_grp = grade_9_full.groupby("school_name")
grade_10_grp = grade_10_full.groupby("school_name")
grade_11_grp = grade_11_full.groupby("school_name")
grade_12_grp = grade_12_full.groupby("school_name")

# Create series' using the grouped dataframes to find the mean math score for each school at each grade level
school_9_math = round(grade_9_grp["math_score"].mean(),2)
school_10_math = round(grade_10_grp["math_score"].mean(),2)
school_11_math = round(grade_11_grp["math_score"].mean(),2)
school_12_math = round(grade_12_grp["math_score"].mean(),2)

# Create a new dataframe using the series' above
school_grade_math = pd.DataFrame({"9th":school_9_math,"10th":school_10_math,"11th":school_11_math,"12th":school_12_math})
school_grade_math

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


In [10]:
# Duplicate the process above using the sorted and grouped dataframes for reading scores instead of math
school_9_reading = round(grade_9_grp["reading_score"].mean(),2)
school_10_reading = round(grade_10_grp["reading_score"].mean(),2)
school_11_reading = round(grade_11_grp["reading_score"].mean(),2)
school_12_reading = round(grade_12_grp["reading_score"].mean(),2)
school_grade_reading = pd.DataFrame({"9th":school_9_reading,"10th":school_10_reading,"11th":school_11_reading,"12th":school_12_reading})
school_grade_reading

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


In [11]:
# Group by the sorted spending ranges column created earlier
funding_grp = complete_df.groupby("Spending Ranges (Per Student)")

#  Use this sorted dataframe to calculate score means and percentage passing numbers
avg_math_funding = round(funding_grp["math_score"].mean(),2)
avg_reading_funding = round(funding_grp["reading_score"].mean(),2)
passing_math_funding = passing_math.groupby("Spending Ranges (Per Student)")
passing_reading_funding = passing_reading.groupby("Spending Ranges (Per Student)")
passing_overall_funding = passing_overall.groupby("Spending Ranges (Per Student)")
perc_math_funding = round((passing_math_funding["Student ID"].count()/funding_grp["Student ID"].count())*100,2)
perc_reading_funding = round((passing_reading_funding["Student ID"].count()/funding_grp["Student ID"].count())*100,2)
perc_overall_funding = round((passing_overall_funding["Student ID"].count()/funding_grp["Student ID"].count())*100,2)

# Create a new dataframe with these sorted values
performance_funding = pd.DataFrame({"Average Math Score": avg_math_funding, \
                              "Average Reading Score": avg_reading_funding, \
                              "% Passing Math": perc_math_funding, \
                              "% Passing Reading": perc_reading_funding, \
                              "% Passing Overall": perc_overall_funding})
performance_funding

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Spending Ranges (Per Student),,,,,
<$585,83.36,83.96,93.70,96.69,90.64
$585-630,79.98,82.31,79.11,88.51,70.94
$630-645,77.82,81.30,70.62,82.60,58.84
$645-680,77.05,81.01,66.23,81.11,53.53


In [12]:
# Group by the sorted size column created earlier
size_grp = complete_df.groupby("School Size")

#  Use this sorted dataframe to calculate score means and percentage passing numbers
avg_math_size = round(size_grp["math_score"].mean(),2)
avg_reading_size = round(size_grp["reading_score"].mean(),2)
passing_math_size = passing_math.groupby("School Size")
passing_reading_size = passing_reading.groupby("School Size")
passing_overall_size = passing_overall.groupby("School Size")
perc_math_size = round((passing_math_size["Student ID"].count()/size_grp["Student ID"].count())*100,2)
perc_reading_size = round((passing_reading_size["Student ID"].count()/size_grp["Student ID"].count())*100,2)
perc_overall_size = round((passing_overall_size["Student ID"].count()/size_grp["Student ID"].count())*100,2)

# Create a new dataframe with these sorted values
performance_size = pd.DataFrame({"Average Math Score": avg_math_size, \
                              "Average Reading Score": avg_reading_size, \
                              "% Passing Math": perc_math_size, \
                              "% Passing Reading": perc_reading_size, \
                              "% Passing Overall": perc_overall_size})
performance_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Size,,,,,
Small (< 1500),83.58,83.89,93.66,96.60,90.37
Medium (1500-3500),80.21,82.53,80.72,88.90,72.89
Large (> 3500),77.07,80.93,66.47,81.11,53.72


In [13]:
# Group by school type
grouped_type = complete_df.groupby("type")

# Create a series for the average math and reading scores by school type
avg_math_type = round(grouped_type["math_score"].mean(),2)
avg_reading_type = round(grouped_type["reading_score"].mean(),2)

# Create dataframes filtered for passing and grouped by school type
passing_math_type = passing_math.groupby("type")
passing_reading_type = passing_reading.groupby("type")
passing_overall_type = passing_overall.groupby("type")

# Calculate the percentages using counts of the created filtered dataframes
perc_math_type = round((passing_math_type["Student ID"].count()/grouped_type["Student ID"].count())*100,2)
perc_reading_type = round((passing_reading_type["Student ID"].count()/grouped_type["Student ID"].count())*100,2)
perc_overall_type = round((passing_overall_type["Student ID"].count()/grouped_type["Student ID"].count())*100,2)

# Designate a new dataframe with all these calculations
performance_type = pd.DataFrame({"Average Math Score": avg_math_type, \
                              "Average Reading Score": avg_reading_type, \
                              "% Passing Math": perc_math_type, \
                              "% Passing Reading": perc_reading_type, \
                              "% Passing Overall": perc_overall_type})
performance_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
type,,,,,
Charter,83.41,83.90,93.70,96.65,90.56
District,76.99,80.96,66.52,80.91,53.70
